In [15]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Conv2D, Dropout, MaxPool2D, Flatten

Using TensorFlow backend.


In [2]:
import pandas as pd
import numpy as np 

In [3]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
submission = pd.read_csv('sample_submission.csv', index_col=0)

In [4]:
train

,rho,650_src,660_src,670_src,680_src,690_src,700_src,710_src,720_src,730_src,...,940_dst,950_dst,960_dst,970_dst,980_dst,990_dst,hhb,hbo2,ca,na
id,,,,,,,,,,,,,,,,,,,,,
0,25,0.37950,0.42993,0.52076,0.57166,0.67818,0.75476,0.83580,0.93623,0.96333,...,NaN,0.000000e+00,NaN,1.067504e-18,5.998949e-18,4.378513e-17,5.59,4.32,8.92,4.29
1,10,0.00000,0.00000,0.01813,0.00000,0.00000,0.01974,0.00321,0.00000,0.00000,...,1.343132e-08,6.112685e-09,2.130547e-09,NaN,9.710091e-09,NaN,0.00,2.83,7.25,4.64
2,25,0.00000,0.03289,0.02416,0.03610,0.05843,0.09015,0.14944,0.18578,0.25584,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.329725e-18,NaN,10.64,3.00,8.40,5.16
3,10,0.27503,0.31281,0.32898,0.41041,0.46587,0.52769,0.64369,0.73562,0.79865,...,2.245998e-10,1.299511e-10,7.782625e-11,NaN,4.088921e-10,NaN,5.67,4.01,5.05,4.35
4,15,1.01521,1.00872,0.98930,0.98874,1.01773,1.01632,1.00009,0.98217,1.01564,...,1.457955e-13,8.769053e-14,NaN,1.330237e-13,NaN,NaN,11.97,4.41,10.78,2.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,15,0.23929,0.30265,0.39929,0.51000,0.64072,0.77328,0.86722,0.95891,0.98998,...,6.788642e-16,0.000000e+00,5.516467e-16,9.690979e-16,1.391635e-15,5.460702e-14,12.68,4.11,12.31,0.10
9996,20,0.02583,0.00946,0.03650,0.01380,0.04093,0.04363,0.03260,0.05282,0.04608,...,1.814274e-16,1.117148e-16,7.352416e-17,NaN,5.825347e-16,4.226436e-15,8.46,4.11,10.46,3.12
9997,10,0.57589,0.62976,0.70571,0.77248,0.85402,0.92796,0.97691,0.98933,1.00493,...,4.828451e-11,3.110419e-11,8.119095e-12,NaN,1.504115e-10,4.003657e-10,9.84,3.20,10.45,2.06


In [5]:
train = train.interpolate()
test = test.interpolate()

In [6]:
train = train.fillna(train.mean())
test = test.fillna(train.mean())

In [7]:
print(train.shape)
print(test.shape)

(10000, 75)
(10000, 71)


In [8]:
train = np.array(train)
x_predict = np.array(test)

In [9]:
x = train[:,:71]
y = train[:,71:]

In [10]:
# 전처리
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x) 
x = scaler.transform(x)
x_predict = scaler.transform(x_predict)

In [11]:
x.shape

(10000, 71)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=43)

In [26]:
model = Sequential()
model.add(Dense(52, input_dim=(71))) 
model.add(Dense(228))
model.add(Dense(356))
model.add(Dropout(0.2))
model.add(Dense(250))
model.add(Dense(164))
model.add(Dropout(0.4))
model.add(Dense(64))
model.add(Dense(4, activation='relu')) 
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 52)                3744      
_________________________________________________________________
dense_21 (Dense)             (None, 228)               12084     
_________________________________________________________________
dense_22 (Dense)             (None, 356)               81524     
_________________________________________________________________
dropout_7 (Dropout)          (None, 356)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 250)               89250     
_________________________________________________________________
dense_24 (Dense)             (None, 164)               41164     
_________________________________________________________________
dropout_8 (Dropout)          (None, 164)              

In [27]:
from sklearn.metrics import mean_absolute_error

In [28]:
model.compile(loss='mae', optimizer='adam', metrics=['mae']) 
model.fit(x_train,y_train, epochs=150, batch_size=1001)
y_pred = model.predict(x_test)
mse = mean_absolute_error(y_pred, y_test)
print(mse)

Epoch 1/150
8000/8000 [==============================] - 0s 24us/step - loss: 4.6125 - mae: 4.6125
Epoch 2/150
8000/8000 [==============================] - 0s 4us/step - loss: 3.8370 - mae: 3.8370
Epoch 3/150
8000/8000 [==============================] - 0s 5us/step - loss: 3.3756 - mae: 3.3756
Epoch 4/150
8000/8000 [==============================] - 0s 4us/step - loss: 2.9636 - mae: 2.9636
Epoch 5/150
8000/8000 [==============================] - 0s 4us/step - loss: 2.3815 - mae: 2.3815
Epoch 6/150
8000/8000 [==============================] - 0s 4us/step - loss: 2.0980 - mae: 2.0980
Epoch 7/150
8000/8000 [==============================] - 0s 5us/step - loss: 1.9512 - mae: 1.9512
Epoch 8/150
8000/8000 [==============================] - 0s 5us/step - loss: 1.8796 - mae: 1.8796
Epoch 9/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.8372 - mae: 1.8372
Epoch 10/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.8175 - mae: 1.8175
Epoch 11/150
8000/

8000/8000 [==============================] - 0s 4us/step - loss: 1.7545 - mae: 1.7545
Epoch 84/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7503 - mae: 1.7503
Epoch 85/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7461 - mae: 1.7461
Epoch 86/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7456 - mae: 1.7456
Epoch 87/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7473 - mae: 1.7473
Epoch 88/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7478 - mae: 1.7478
Epoch 89/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7454 - mae: 1.7454
Epoch 90/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7431 - mae: 1.7431
Epoch 91/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7436 - mae: 1.7436
Epoch 92/150
8000/8000 [==============================] - 0s 4us/step - loss: 1.7424 - mae: 1.7424
Epoch 93/150
8000/8000 

In [13]:
from sklearn.datasets import load_boston
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.utils.testing import all_estimators  # sklearn 0.20.1에서만 돌아감요
import warnings
warnings.filterwarnings('ignore')

In [14]:
allAlgorithms = all_estimators(type_filter='regressor')

for (name, algorithm) in allAlgorithms :
    model = algorithm()
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(name, '의 mae = ',  mean_absolute_error(y_pred, y_test))

ValueError: bad input shape (8000, 4)

In [13]:
submission
# 와우 4개를 맞추는거네...?
# 심지어 결측치도 있음
# 일단 어떤 데이터인지 보고 결측치 처리하는거 고려해봐야해 왜냐면 test셋에도 결측치가 있기 때문임
# 지금으로써는 결측치 많은 칼럼은 아예 없애버리고 싶음 왜냐면 이미 칼럼이 너무 많아
# 대회는 import가 전부다 ^^@

,hhb,hbo2,ca,na
id,,,,
10000,0,0,0,0
10001,0,0,0,0
10002,0,0,0,0
10003,0,0,0,0
10004,0,0,0,0
...,...,...,...,...
19995,0,0,0,0
19996,0,0,0,0
19997,0,0,0,0
